## ÁREA VERDE URBANA PER CÁPITA

In [1]:
#CARGAR LIBRERIAS
libraries <- c("tidyverse",
               "sf",  
               "DBI", 
               "RSQLite", 
               "utils", 
               "readxl")

installed_packages <- rownames(installed.packages())
libraries_to_install <- setdiff(libraries, installed_packages)

if (length(libraries_to_install) > 0) {
  install.packages(libraries_to_install)
}

lapply(libraries, library, character.only = TRUE)

print ("librerias cargadas")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Linking to GEOS 3.13.0, GDAL 3.10.1, PROJ 9.5.1; sf_use_s2() is TRUE



[[1]]
 [1] "lubridate" "forcats"   "stringr"   "dplyr"     "purrr"     "readr"    
 [7] "tidyr"     "tibble"    "ggplot2"   "tidyverse" "stats"     "graphics" 
[13] "grDevices" "utils"     "datasets"  "methods"   "base"     

[[2]]
 [1] "sf"        "lubridate" "forcats"   "stringr"   "dplyr"     "purrr"    
 [7] "readr"     "tidyr"     "tibble"    "ggplot2"   "tidyverse" "stats"    
[13] "graphics"  "grDevices" "utils"     "datasets"  "methods"   "base"     

[[3]]
 [1] "DBI"       "sf"        "lubridate" "forcats"   "stringr"   "dplyr"    
 [7] "purrr"     "readr"     "tidyr"     "tibble"    "ggplot2"   "tidyverse"
[13] "stats"     "graphics"  "grDevices" "utils"     "datasets"  "methods"  
[19] "base"     

[[4]]
 [1] "RSQLite"   "DBI"       "sf"        "lubridate" "forcats"   "stringr"  
 [7] "dplyr"     "purrr"     "readr"     "tidyr"     "tibble"    "ggplot2"  
[13] "tidyverse" "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[19] "methods"   "base"     

[[5]]
 [1] "RSQLite"   "DBI"       "sf"        "lubridate" "forcats"   "stringr"  
 [7] "dplyr"     "purrr"     "readr"     "tidyr"     "tibble"    "ggplot2"  
[13] "tidyverse" "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[19] "methods"   "base"     

[[6]]
 [1] "readxl"    "RSQLite"   "DBI"       "sf"        "lubridate" "forcats"  
 [7] "stringr"   "dplyr"     "purrr"     "readr"     "tidyr"     "tibble"   
[13] "ggplot2"   "tidyverse" "stats"     "graphics"  "grDevices" "utils"    
[19] "datasets"  "methods"   "base"

[1] "librerias cargadas"


**Consideraciones Generales del Área Verde Definida**

La definición del área verde parte de la *Ley Ambiental de la Ciudad de México*, publicada en la Gaceta Oficial de la Ciudad de México el *18 de julio de 2024*, que en su *artículo 4, fracción VIII*, establece como *Áreas Verdes* a:

> "Toda superficie cubierta de vegetación, natural o inducida, que se localice en la Ciudad de México."

Fuente:  
CONABIO (2024). *Áreas verdes 2017–2018 de la Ciudad de México*.  Escala: 1:4,000, Edición: 1,  Vigencia: 2020–2023. Proyecto: *Biodiversidad Urbana*, Ciudad de México.

*Mantenimiento:* No

In [2]:
directorio <- "C:/Users/brenp/Desktop/PGOTDU_CDMX/ESPACIO PUBLICO"
setwd(directorio)
getwd()

[1] "C:/Users/brenp/Desktop/PGOTDU_CDMX/ESPACIO PUBLICO"

In [ ]:
##PRINCIPALES RESULTADOS POR MANZANA CENSO DE POBLACIÓN Y VIVIENDA 2020
url <- "https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/ageb_manzana/RESAGEBURB_09_2020_xlsx.zip"
temp_zip <- tempfile(fileext = ".zip")
temp_dir <- tempdir()
options(timeout = 800)
download.file(url, temp_zip, mode = "wb")
unzip(temp_zip, exdir = temp_dir)
RESAGEBURB_09XLSX20_BASE <- list.files(temp_dir, pattern ="RESAGEBURB_09XLSX20.xlsx", full.names = TRUE, recursive = TRUE)
RESAGEBURB_09XLSX20<- readxl::read_excel(RESAGEBURB_09XLSX20_BASE)|>
  mutate(CVEGEO = paste0(ENTIDAD),
         CVE_MUNI = paste0(ENTIDAD, MUN))|>
  mutate_all(~ gsub("\\*", "", .)) |> 
  mutate(across(9:230, ~ as.numeric(., na.rm = TRUE)))|>
  filter(NOM_MUN == "Total de la entidad Ciudad de México")|>
  select(CVEGEO, POBTOT)



In [4]:
#ZONA URBANA DEFINIDA CON LAS AGEBS MARCO GEOESTADÍSTICO CDMX 2020
url <- "https://www.inegi.org.mx/contenidos/productos/prod_serv/contenidos/espanol/bvinegi/productos/geografia/marcogeo/889463807469/09_ciudaddemexico.zip"
temp_zip <- tempfile(fileext = ".zip")     
temp_dir <- tempdir()                       
options(timeout = 1000)                      
download.file(url, temp_zip, mode = "wb")
related_files <- paste0("conjunto_de_datos/09a", c(".shp", ".shx", ".dbf", ".prj"))
unzip(temp_zip, files = related_files, exdir = temp_dir)
shp_file_path <- file.path(temp_dir, related_files[1])
zona_urbana_ageb_2020 <- st_read(shp_file_path, quiet = TRUE)|>
  st_transform(crs = 32614)|>
  mutate(area = st_area(geometry))|>
  group_by(CVE_ENT)|>
  summarise(area_urbana = sum(area))


In [5]:
##Áreas verdes de la Ciudad de México (2017-2018)----
# url <- "http://www.conabio.gob.mx/informacion/gis/maps/ccl/avcdmxgw_c.zip"
# options(timeout = 5000)
# download.file(url, 'avcdmxgw_c.zip', mode = "wb")

# zip::unzip("avcdmxgw_c.zip", exdir = "avcdmx_1718")

acvdmx_gdbs<-st_read("avcdmx_1718/avcdmx_1718.gdb")|>
  mutate(CVEGEO = CVE_ENT)|>
  st_intersection(zona_urbana_ageb_2020)|> 
  st_drop_geometry()|>
  as.data.frame()|>
  group_by(CVEGEO)|>
  summarise(area_total = sum(Shape_Area))

head(acvdmx_gdbs)

Reading layer `avcdmx_1718' from data source 
  `C:\Users\brenp\Desktop\PGOTDU_CDMX\ESPACIO PUBLICO\avcdmx_1718\avcdmx_1718.gdb' 
  using driver `OpenFileGDB'


Warning message in CPL_read_ogr(dsn, layer, query, as.character(options), quiet, :
"GDAL Message 1: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined"


Simple feature collection with 2129346 features and 11 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 461669.7 ymin: 2106221 xmax: 506246 ymax: 2166416
Projected CRS: WGS 84 / UTM zone 14N


Warning message:
"attribute variables are assumed to be spatially constant throughout all geometries"


CVEGEO,area_total
<chr>,<dbl>
09,676939891


In [6]:
##ÁREA VERDE PER CAPITA
av_pc_2018<-acvdmx_gdbs|>
  left_join(RESAGEBURB_09XLSX20, by = "CVEGEO")|>
  mutate(AV_PC_18 = round(area_total/POBTOT,2))

In [7]:
print(paste("Área verde per cápita: ", av_pc_2018$AV_PC_18, "m2/hab"))

[1] "Área verde per cápita:  73.5 m2/hab"
